In [2]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.autoencoder import autoencoder as ae
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder as onehot
from imblearn.over_sampling import RandomOverSampler

# path = "/media/jahan/solo/"
path = "/Volumes/JAHAN/"
os.chdir(path)

# prep data
def prep(csv):   
    labels = pd.read_csv(csv)
    labels = labels[labels["Frontal/Lateral"] == "Frontal"]
    normal = labels[labels["No Finding"] == 1]
    normal = normal.Path
    abnormal = labels[labels["No Finding"] != 1]
    abnormal = abnormal.Path
    print(len(normal),len(abnormal))

    x_a = pd.concat([normal,abnormal]).values
    x_b = pd.concat([normal.sample(frac=1),abnormal.sample(frac=1)]).values
    x_c = pd.concat([abnormal.sample(n=len(normal)),normal.sample(n=len(abnormal),replace=True)]).values
    
    x = np.stack((x_a,x_b,x_c), axis=-1)
    
    y = np.full(len(normal),0)
    y = np.append(y, np.full(len(abnormal),1))
    
    sm = RandomOverSampler(random_state=42)
    X_res, y_res = sm.fit_resample(x, y)
    
    print(X_res.shape)
    
    X = np.stack((X_res[:,0], X_res[:,1], X_res[:,2], y_res), axis=-1)
    np.random.shuffle(X)
    
    return X[:,0], X[:,1], X[:,2], X[:,3]

x_a,x_b,x_c, y = prep("CheXpert-v1.0-small/train.csv")

print(y)

config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.9
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


# save_path = "/home/jahan/Documents/CXR_CNN/output/triple"
save_path = "/Users/jahan/Documents/research/CXR_CNN/output"
print(x_a.size)

FileNotFoundError: [Errno 2] File b'CheXpert-v1.0-small/train.csv' does not exist: b'CheXpert-v1.0-small/train.csv'

In [2]:
# strategy = tf.distribute.MirroredStrategy()
strategy = tf.distribute.experimental.CentralStorageStrategy()
with strategy.scope():
    encoder = ae(save_path=save_path)
    encoder.build(load="/triple.ckpt")
#     encoder.build()

W1222 18:53:00.460455 140067193993024 deprecation.py:506] From /home/jahan/miniconda3/envs/ml_gpu/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 416         input_1[0][0]                    
                                                                 input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
                                                                 conv2d[1][0]                 

In [5]:
with strategy.scope():
    history = encoder.train(a_paths=x_a, b_paths=x_b, c_paths=x_c, epochs=1)

W1222 19:56:51.861301 140025647748864 deprecation.py:323] From /home/jahan/miniconda3/envs/ml_gpu/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


  1456/400000 [..............................] - ETA: 9:53:17 - loss: 0.0254 - binary_accuracy: 0.9916

KeyboardInterrupt: 

In [6]:
labels = pd.read_csv("CheXpert-v1.0-small/valid.csv")
labels = labels[labels["Frontal/Lateral"] == "Frontal"]
normal = labels[labels["No Finding"] == 1]
normal = normal.Path
abnormal = labels[labels["No Finding"] != 1]
abnormal = abnormal.Path
print(len(normal),len(abnormal))

x_a = pd.concat([normal,abnormal]).values
x_b = pd.concat([normal.sample(frac=1),abnormal.sample(frac=1)]).values
x_c = pd.concat([abnormal.sample(n=len(normal)),normal.sample(n=len(abnormal),replace=True)]).values

y = np.full(len(normal),0)
y = np.append(y, np.full(len(abnormal),1))

print(x_a.size)

26 176
202


In [ ]:
probs = encoder.predict_proba(x_a,x_b,x_c, y)
print(probs)

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y, probs)

print(auc)